In [1]:
%load_ext autoreload
%autoreload 2

In [36]:
import os
import pandas as pd
import numpy as np

from glob import glob
import altair as alt
from tqdm.auto import tqdm
import imageio

from tools import get_all_files

In [3]:
data_root = '../2023_11_03__17_49_42_hard_mincost_RF30_P1e4_Won_Gon'

In [4]:
# Get all output files
files = get_all_files(data_root)
files['year'] = files['year'].astype(int)

files_selc = files.query('year <= 2050')
print(files_selc['catetory'].unique())

['GHG' 'dvar' 'ammap' 'lmmap' 'lumap_separate' 'lumap' 'water'
 'cross_table' 'quantity']


In [5]:
files_selc

,year,catetory,base_name,base_ext,path
0,2010,GHG,GHG_emissions,.csv,../2023_11_03__17_49_42_hard_mincost_RF30_P1e4...
1,2010,GHG,GHG_emissions_separate_agricultural_landuse,.csv,../2023_11_03__17_49_42_hard_mincost_RF30_P1e4...
2,2010,GHG,GHG_emissions_separate_agricultural_management,.csv,../2023_11_03__17_49_42_hard_mincost_RF30_P1e4...
3,2010,GHG,GHG_emissions_separate_no_ag_reduction,.csv,../2023_11_03__17_49_42_hard_mincost_RF30_P1e4...
4,2010,GHG,GHG_emissions_separate_transition_penalty,.csv,../2023_11_03__17_49_42_hard_mincost_RF30_P1e4...
...,...,...,...,...,...
2482,2050,cross_table,switches-irrstat,.csv,../2023_11_03__17_49_42_hard_mincost_RF30_P1e4...
2483,2050,cross_table,switches-lmmap,.csv,../2023_11_03__17_49_42_hard_mincost_RF30_P1e4...
2484,2050,cross_table,switches-lumap,.csv,../2023_11_03__17_49_42_hard_mincost_RF30_P1e4...
2485,2050,cross_table,switches-precision-agriculture-amstat,.csv,../2023_11_03__17_49_42_hard_mincost_RF30_P1e4...


## GHG emissions

In [6]:
# Get only GHG_seperate files
GHG_files = files_selc.query('catetory == "GHG" and base_name != "GHG_emissions" ').reset_index(drop=True)
GHG_files['GHG_sum_t'] = GHG_files['path'].apply(lambda x: pd.read_csv(x,index_col=0).loc['SUM','SUM'])
GHG_files = GHG_files.replace({'base_name': {'GHG_emissions_separate_agricultural_landuse': 'Agricultural Landuse',
                                             'GHG_emissions_separate_agricultural_management': 'Agricultural Management',
                                             'GHG_emissions_separate_no_ag_reduction': 'Non-Agricultural Landuse',
                                             'GHG_emissions_separate_transition_penalty': 'Transition Penalty'}})

In [7]:
GHG_files = GHG_files.reset_index(drop=True).sort_values(['year','GHG_sum_t'])
GHG_files['GHG_sum_Mt'] = GHG_files['GHG_sum_t'] / 1e6
GHG_files

,year,catetory,base_name,base_ext,path,GHG_sum_t,GHG_sum_Mt
1,2010,GHG,Agricultural Management,.csv,../2023_11_03__17_49_42_hard_mincost_RF30_P1e4...,0.000000e+00,0.000000
2,2010,GHG,Non-Agricultural Landuse,.csv,../2023_11_03__17_49_42_hard_mincost_RF30_P1e4...,0.000000e+00,0.000000
3,2010,GHG,Transition Penalty,.csv,../2023_11_03__17_49_42_hard_mincost_RF30_P1e4...,0.000000e+00,0.000000
0,2010,GHG,Agricultural Landuse,.csv,../2023_11_03__17_49_42_hard_mincost_RF30_P1e4...,3.682068e+07,36.820685
6,2011,GHG,Non-Agricultural Landuse,.csv,../2023_11_03__17_49_42_hard_mincost_RF30_P1e4...,-1.090027e+08,-109.002744
...,...,...,...,...,...,...,...
156,2049,GHG,Agricultural Landuse,.csv,../2023_11_03__17_49_42_hard_mincost_RF30_P1e4...,3.626533e+07,36.265328
162,2050,GHG,Non-Agricultural Landuse,.csv,../2023_11_03__17_49_42_hard_mincost_RF30_P1e4...,-1.056718e+08,-105.671840
161,2050,GHG,Agricultural Management,.csv,../2023_11_03__17_49_42_hard_mincost_RF30_P1e4...,-2.062908e+06,-2.062908
163,2050,GHG,Transition Penalty,.csv,../2023_11_03__17_49_42_hard_mincost_RF30_P1e4...,0.000000e+00,0.000000


In [8]:
# Convert to wide format for plotting in Flourish
GHG_files_wide = GHG_files.pivot(index='year', columns='base_name', values='GHG_sum_Mt').reset_index()
GHG_files_wide.insert(1, 'Net Emission', GHG_files_wide[['Agricultural Landuse','Agricultural Management','Non-Agricultural Landuse']].sum(axis=1))
GHG_files_wide.to_csv('output/GHG_files_wide.csv', index=False)

#### Total emissions -- Column chart

In [9]:
# Create a base chart with the necessary transformations and encodings
base_chart = alt.Chart(GHG_files).transform_calculate(
    GHG_sum_Mt = "datum.GHG_sum_t/1000000"
).encode(
    x=alt.X('year:O',axis=alt.Axis(title="Year", labelAngle=-90)),  # Treat year as an ordinal data type
    tooltip=[alt.Tooltip('base_name', title='GHG Category'),
             alt.Tooltip('GHG_sum_Mt:Q', title='Emissions (Mt CO2e)')]
).properties(
    width=600,
    height=400
)



# Create a column chart with the base chart
column_chart = base_chart.mark_bar().encode(
    color=alt.Color('base_name:N',legend=alt.Legend(
                                            title="GHG Category",
                                            orient='none',
                                            legendX=130, legendY=-40,
                                            direction='horizontal',
                                            titleAnchor='middle')),  
    y=alt.Y('GHG_sum_Mt:Q',title='Emissions (Mt CO2e)'),  # Treat GHA_accumulative as a quantitative field
)



# Combine the layers into a final chart
final_chart = alt.layer(
    column_chart,
).properties(
    width=800,
    height=450
)



final_chart


alt.LayerChart(...)

### Agricultural Land use Emissions

#### Emissions by LUCC -- Stack column

In [10]:
LU_crops = ['Apples','Citrus','Cotton','Grapes','Hay','Nuts','Other non-cereal crops',
            'Pears','Plantation fruit','Rice','Stone fruit','Sugar','Summer cereals',
            'Summer legumes','Summer oilseeds','Tropical stone fruit','Vegetables',
            'Winter cereals','Winter legumes','Winter oilseeds']

LU_lvstk = ['Beef - modified land','Beef - natural land','Dairy - modified land',
             'Dairy - natural land','Sheep - modified land','Sheep - natural land']

In [11]:
# Get GHG emissions from ag lucc
GHG_ag_lucc = GHG_files.query('base_name == "Agricultural Landuse"').reset_index(drop=True)
GHG_ag_lucc.head(3)

,year,catetory,base_name,base_ext,path,GHG_sum_t,GHG_sum_Mt
0,2010,GHG,Agricultural Landuse,.csv,../2023_11_03__17_49_42_hard_mincost_RF30_P1e4...,3.682068e+07,36.820685
1,2011,GHG,Agricultural Landuse,.csv,../2023_11_03__17_49_42_hard_mincost_RF30_P1e4...,3.887889e+07,38.878895
2,2012,GHG,Agricultural Landuse,.csv,../2023_11_03__17_49_42_hard_mincost_RF30_P1e4...,3.821777e+07,38.217768


In [12]:
# Read GHG emissions of ag lucc
GHG_ag_lucc_CSVs = []
for _,row in tqdm(GHG_ag_lucc.iterrows(),total=GHG_ag_lucc.shape[0]):
    csv = pd.read_csv(row['path'],index_col=0,header=[0,1,2]).drop('SUM',axis=1)

    csv_crop = csv[[True if i in LU_crops else False for i in  csv.index]]
    csv_crop.index = pd.MultiIndex.from_product(([row['year']], csv_crop.index, ['Crop']))

    csv_lvstk = csv[[True if i in LU_lvstk else False for i in  csv.index]]
    csv_lvstk.index = pd.MultiIndex.from_product(([row['year']], csv_lvstk.index, ['Livestock']))

    csv = pd.concat([csv_crop,csv_lvstk],axis=1)
    GHG_ag_lucc_CSVs.append(csv)

  0%|          | 0/41 [00:00<?, ?it/s]

In [13]:
# get the GHG_crop_lvstk_total
GHG_ag_df = pd.concat(GHG_ag_lucc_CSVs,axis=0)
GHG_ag_df.columns = GHG_ag_df.columns.droplevel([0])
GHG_ag_crop_lvstk_total = GHG_ag_df.groupby(level=[0,2]).sum().sum(axis=1).reset_index()

GHG_ag_crop_lvstk_total.columns = ['year','Landuse type','GHG emissions (t CO2e)']
GHG_ag_crop_lvstk_total['GHG emissions (Mt CO2e)'] = GHG_ag_crop_lvstk_total['GHG emissions (t CO2e)'] / 1e6

GHG_ag_crop_lvstk_total.head(3)

,year,Landuse type,GHG emissions (t CO2e),GHG emissions (Mt CO2e)
0,2010,Crop,1.773589e+07,17.735888
1,2010,Livestock,1.908480e+07,19.084797
2,2011,Crop,1.303577e+07,13.035769


In [14]:
# make the df wide format
GHG_ag_crop_lvstk_total_wide = GHG_ag_crop_lvstk_total.pivot(index='year',
                                                             columns='Landuse type',
                                                             values='GHG emissions (Mt CO2e)').reset_index()
GHG_ag_crop_lvstk_total_wide.columns = GHG_ag_crop_lvstk_total_wide.columns.tolist()
GHG_ag_crop_lvstk_total_wide.to_csv('output/GHG_ag_crop_lvstk_total_wide.csv',index=False)

In [15]:
base_chart = alt.Chart(GHG_ag_crop_lvstk_total).encode(
    x=alt.X('year:O',axis=alt.Axis(title="Year", labelAngle=-90)),  # Treat year as an ordinal data type
    tooltip=[alt.Tooltip('Landuse type', title='Landuse type'),
             alt.Tooltip('GHG emissions (Mt CO2e):Q', title='Emissions (Mt CO2e)')]
).properties(
    width=600,
    height=400
)

column_chart = base_chart.mark_bar().encode(
    color=alt.Color('Landuse type:N',legend=alt.Legend(
                                            title="Landuse type",
                                            orient='none',
                                            legendX=350, legendY=-40,
                                            direction='horizontal',
                                            titleAnchor='middle')),  
    y=alt.Y('GHG emissions (Mt CO2e):Q',title='Emissions (Mt CO2e)'),  # Treat GHA_accumulative as a quantitative field
)

final_chart = alt.layer(
    column_chart,
).properties(
    width=800,
    height=450
)

final_chart

alt.LayerChart(...)

#### Emissions by dry/irr -- Stack columns

In [16]:
# get the GHG_crop_lvstk_total
GHG_ag_lm_total = GHG_ag_df.groupby(level=[0]).sum()
GHG_ag_lm_total_wide = GHG_ag_lm_total.groupby(level=0,axis=1).sum().reset_index()
GHG_ag_lm_total_wide.columns = ['year','dry','irr']
GHG_ag_lm_total_wide['year'] = GHG_ag_lm_total_wide['year'].astype(int)
GHG_ag_lm_total_wide[['dry','irr']] = GHG_ag_lm_total_wide[['dry','irr']] / 1e6

GHG_ag_lm_total_wide.to_csv('output/GHG_ag_lm_total_wide.csv',index=False)
GHG_ag_lm_total_wide.head(3)

,year,dry,irr
0,2010,30.126696,6.693989
1,2011,35.701458,3.151075
2,2012,35.340035,2.869060


In [17]:
# make the long format table
GHG_ag_lm_total = GHG_ag_lm_total_wide.melt(id_vars='year',value_vars=['dry','irr'],var_name='irrigation')
GHG_ag_lm_total.columns = ['year','irrigation','GHG emissions (Mt CO2e)']
GHG_ag_lm_total.head(3)

,year,irrigation,GHG emissions (Mt CO2e)
0,2010,dry,30.126696
1,2011,dry,35.701458
2,2012,dry,35.340035


In [18]:
base_chart = alt.Chart(GHG_ag_lm_total).encode(
    x=alt.X('year:O',axis=alt.Axis(title="Year", labelAngle=-90)),  # Treat year as an ordinal data type
    tooltip=[alt.Tooltip('irrigation', title='Irrigation'),
             alt.Tooltip('GHG emissions (Mt CO2e):Q', title='Emissions (Mt CO2e)')]
).properties(
    width=600,
    height=400
)



column_chart = base_chart.mark_bar().encode(
    color=alt.Color('irrigation:N',legend=alt.Legend(
                                                        title="Irrigation",
                                                        orient='none',
                                                        legendX=350, legendY=-40,
                                                        direction='horizontal',
                                                        titleAnchor='middle')),  
    y=alt.Y('GHG emissions (Mt CO2e):Q',
        title='Emissions (Mt CO2e)')
    

)



final_chart = alt.layer(
    column_chart,
).properties(
    width=800,
    height=450
)

final_chart



alt.LayerChart(...)

#### Emissions by lu-lm cmobined

In [19]:
GHG_ag_lu_lm = GHG_ag_df.groupby(level=[1,2]).mean()
GHG_ag_lu_lm = GHG_ag_lu_lm.groupby(level=0,axis=1).sum().reset_index()
GHG_ag_lu_lm.columns = ['Land use','Landuse type','dry','irr']
GHG_ag_lu_lm = GHG_ag_lu_lm.melt(id_vars=['Land use','Landuse type'],value_vars=['dry','irr'],
                                    var_name='Irrigation',value_name='GHG emissions (t CO2e)')


GHG_ag_lu_lm['GHG emissions (Mt CO2e)'] = GHG_ag_lu_lm['GHG emissions (t CO2e)'] / 1e6
GHG_ag_lu_lm.drop('GHG emissions (t CO2e)',axis=1,inplace=True)
GHG_ag_lu_lm = GHG_ag_lu_lm[GHG_ag_lu_lm['GHG emissions (Mt CO2e)'] != 0]

GHG_ag_lu_lm.to_csv('output/GHG_ag_lu_lm.csv',index=False)

In [20]:
GHG_ag_lu_lm.head(3)

,Land use,Landuse type,Irrigation,GHG emissions (Mt CO2e)
0,Beef - modified land,Livestock,dry,3.251527
1,Beef - natural land,Livestock,dry,2.509195
3,Cotton,Crop,dry,0.445986


In [21]:
alt.Chart(GHG_ag_lu_lm).mark_bar().encode(
    column="Landuse type:O",
    tooltip=[alt.Tooltip('Irrigation:O', title='Irrigation'),
            alt.Tooltip('GHG emissions (Mt CO2e):Q', title='Emissions (Mt CO2e)')],
    x="GHG emissions (Mt CO2e):Q",
    y="Land use:O",
    color="Irrigation:O",
).properties(width=220)

alt.Chart(...)

#### Emissions by lu-sources combined

In [22]:
GHG_ag_lu_source = GHG_ag_df.groupby(level=[0,1]).mean()
GHG_ag_lu_source.columns = ["+".join(i) for i in GHG_ag_lu_source.columns.tolist()]
GHG_ag_lu_source = GHG_ag_lu_source.reset_index()
GHG_ag_lu_source.columns = ['Year','Land Use'] + GHG_ag_lu_source.columns.tolist()[2:]
GHG_ag_lu_source = GHG_ag_lu_source.melt(id_vars=['Year','Land Use'],
                                         value_vars=GHG_ag_lu_source.columns.tolist()[2:],
                                         value_name='GHG emissions (t CO2e)')


GHG_ag_lu_source['GHG emissions (Mt CO2e)'] = GHG_ag_lu_source['GHG emissions (t CO2e)'] / 1e6
GHG_ag_lu_source[['Irrigation','Sources']] = GHG_ag_lu_source['variable'].str.split('+',expand=True)
GHG_ag_lu_source.drop(['GHG emissions (t CO2e)','variable'],axis=1,inplace=True)
GHG_ag_lu_source = GHG_ag_lu_source.query('`GHG emissions (Mt CO2e)` > 0').reset_index(drop=True)

GHG_ag_lu_source.to_csv('output/GHG_ag_lu_source.csv',index=False)

In [23]:
GHG_ag_lu_source.head(3)

,Year,Land Use,GHG emissions (Mt CO2e),Irrigation,Sources
0,2010,Cotton,0.019594,dry,TCO2E_CHEM_APPL
1,2010,Hay,0.018938,dry,TCO2E_CHEM_APPL
2,2010,Nuts,0.018463,dry,TCO2E_CHEM_APPL


In [46]:
gif.frame
def plot_GHG_ag_lu_source(total_df,year):
    df = total_df.query(f'Year == {year}')
    plot = alt.Chart(df).mark_circle().encode(
        alt.X('Land Use:O'),
        alt.Y('Sources:O'),
        alt.Color('Irrigation:O'),
        size='GHG emissions (Mt CO2e):Q'
    )
    plot.save(f'output/figs/GHG_ag_lu_source_{year}.png')
    return imageio.v2.imread(f'output/figs/GHG_ag_lu_source_{year}.png')


In [47]:
# Construct "frames"
frames = [plot_GHG_ag_lu_source(GHG_ag_lu_source,i) for i in GHG_ag_lu_source['Year'].unique()]

In [ ]:
alt.Chart(GHG_ag_lu_source).mark_bar().encode(
    facet=alt.Facet('Land Use:O', columns=8),
    x="GHG emissions (Mt CO2e):Q",
    y="Sources:O",
    color="Irrigation:O",
).properties(width=100,height=200)

alt.Chart(...)